# Modelo celular autómata

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [2]:
def animar_solucion(X, U, NT, T, title, doc_name, step=None):
    """
    Función para crear una animación y guardarla como gif.
    Utiliza la función FuncAnimation de matplotlib.
    
    X, U son listas que contienen arreglos a graficar. Deben tener el mismo tamaño.
    """
    # crea plot
    fig, ax = plt.subplots()
    ax.set_ylim(-0.1, 1.1)
    lines = []
    
    for i in range(len(U)):
        u = U[i]
        x = X[i]
        
        # genera un plot inicial para cada par de datos de entrada
        line, = ax.plot(x, u[0])
        lines += [line]

    # Parametros animacion
    niter = NT
    step = T
    
    if step is not None:
        step = step

    # Funcion para animar
    def animate(i, X, U):
        
        # Pasa por cada par de datos de entrada
        for j in range(len(U)):
            u = U[j]
            x = X[j]
            
            # Actualiza grafico
            lines[j].set_data(x, u[i])

        ax.set_title(title + " t={}".format(i))
        return ax

    # Animación
    ani = animation.FuncAnimation(fig, animate, frames=range(1,niter,step), interval=100, 
                                    fargs = (X, U), repeat=True)

    ani.save(doc_name)